<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
import requests
# Use requests.get() to request the HTML page
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Success! Page retrieved.")
else:
    print(f"Error: Failed to retrieve page. Status code: {response.status_code}")
# assign the response to a object

Success! Page retrieved.


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
import requests
from bs4 import BeautifulSoup

# Static URL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Get the HTML content
response = requests.get(static_url)

# Check for successful response
if response.status_code == 200:
    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.content, 'html.parser')  # Use 'html.parser' or 'lxml'
    print("BeautifulSoup object created successfully")
    # You can now use soup to parse the HTML
    # For example, to print the title of the page:
    print(soup.title)
else:
    print(f"Error: Failed to retrieve page. Status code: {response.status_code}")

BeautifulSoup object created successfully
<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Use soup.title attribute
import requests
from bs4 import BeautifulSoup

# Static URL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Get the HTML content
response = requests.get(static_url)

# Check for successful response
if response.status_code == 200:
    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.content, 'html.parser')

    # Print the page title
    print(soup.title.string)  # Access the string within the title tag
else:
    print(f"Error: Failed to retrieve page. Status code: {response.status_code}")

List of Falcon 9 and Falcon Heavy launches - Wikipedia


In [ ]:
data_falcon9 = launch_df[launch_df['BoosterVersion'] == 'Falcon 9']
num_falcon9_launches = len(data_falcon9)
print(f"Number of Falcon 9 launches: {num_falcon9_launches}")

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
import requests
from bs4 import BeautifulSoup

static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all tables on the page
    html_tables = soup.find_all('table')

    # Print the number of tables found (for verification)
    print(f"Number of tables found: {len(html_tables)}")

    #Optional: print the first table to see what it contains
    #print(html_tables[0])

else:
    print(f"Error: Failed to retrieve page. Status code: {response.status_code}")


Number of tables found: 26


Starting from the third table is our target table contains the actual launch records.


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [9]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
import requests
from bs4 import BeautifulSoup

static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    html_tables = soup.find_all('table')

    # Select the third table (index 2)
    first_launch_table = html_tables[2]

    # Extract column names
    column_names = []
    for th in first_launch_table.find_all('th'):
        name = th.text.strip()  # Extract text and remove leading/trailing spaces
        if name and len(name) > 0:  # Check if the name is not empty
            column_names.append(name)

    print("Column Names:", column_names)

else:
    print(f"Error: Failed to retrieve page. Status code: {response.status_code}")

Column Names: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']


Check the extracted column names


In [10]:
print(column_names)

['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [ ]:
import requests
from bs4 import BeautifulSoup

static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    html_tables = soup.find_all('table')
    first_launch_table = html_tables[2]

    column_names = []
    for th in first_launch_table.find_all('th'):
        name = th.text.strip()
        if name:
            column_names.append(name)

    launch_dict = dict.fromkeys(column_names)

    # Remove irrelevant column (handle potential KeyError)
    try:
        del launch_dict['Date and time ( )']
    except KeyError:
        print("Key 'Date and time ( )' not found in column names.")

    # Initialize dictionary values as empty lists
    launch_dict['Flight No.'] = []
    launch_dict['Launch site'] = []
    launch_dict['Payload'] = []
    launch_dict['Payload mass'] = []
    launch_dict['Orbit'] = []
    launch_dict['Customer'] = []
    launch_dict['Launch outcome'] = []
    launch_dict['Version Booster'] = []
    launch_dict['Booster landing'] = []
    launch_dict['Date'] = []
    launch_dict['Time'] = []
    print(launch_dict)
else:
    print(f"Error: Failed to retrieve page. Status code: {response.status_code}")

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [18]:
import requests
from bs4 import BeautifulSoup

def date_time(element):
    """Extracts date and time from a table cell element.

    Args:
        element (bs4.element.Tag): The table cell element containing date and time.

    Returns:
        list: A list containing the extracted date and time strings.
    """
    return [el.strip(',') for el in element.text.split(' ')]  # Split, strip commas, return as list

def booster_version(element):
    """Extracts booster version from a table cell element.

    Args:
        element (bs4.element.Tag): The table cell element containing booster version.

    Returns:
        str: The extracted booster version string, or None if not found.
    """
    if element.a:  # Check if there's an anchor tag
        return element.a.string  # Extract text from anchor tag
    else:
        return None  # No version found

def get_mass(element):
    """Extracts payload mass from a table cell element.

    Args:
        element (bs4.element.Tag): The table cell element containing payload mass.

    Returns:
        str: The extracted payload mass string, or None if not found.
    """
    mass_text = element.text.strip()  # Extract and strip text
    if mass_text:
        return mass_text.replace('kg', '')  # Remove unit if it exists
    else:
        return None  # No mass found

def landing_status(element):
    """Extracts launch outcome from a table cell element.

    Args:
        element (bs4.element.Tag): The table cell element containing launch outcome.

    Returns:
        str: The extracted launch outcome string, or None if not found.
    """
    try:
        # Attempt to find a specific class (may change in the future)
        landing_text = element.find('div', class_='mw-collapsible-text').text.strip()
        return landing_text
    except AttributeError:
        return None  # No landing status found

static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    html_tables = soup.find_all('table', "wikitable plainrowheaders collapsible")
    launch_dict = dict.fromkeys(column_names)

    # Remove irrelevant column (handle potential KeyError)
    try:
        del launch_dict['Date and time ( )']
    except KeyError:
        print("Key 'Date and time ( )' not found in column names.")

    # Initialize dictionary values as empty lists
    launch_dict['Flight No.'] = []
    launch_dict['Launch site'] = []
    launch_dict['Payload'] = []
    launch_dict['Payload mass'] = []
    launch_dict['Orbit'] = []
    launch_dict['Customer'] = []
    launch_dict['Launch outcome'] = []
    launch_dict['Version Booster'] = []
    launch_dict['Booster landing'] = []
    launch_dict['Date'] = []
    launch_dict['Time'] = []

    extracted_row = 0
    for table_number, table in enumerate(html_tables):
        for rows in table.find_all("tr"):
            if rows.th:
                if rows.th.string:
                    flight_number = rows.th.string.strip()
                    flag = flight_number.isdigit()
            else:
                flag = False

            row = rows.find_all('td')
            if flag:
                extracted_row += 1

                # Flight Number
                launch_dict['Flight No.'].append(flight_number)

                # Date and Time
                datetime_list = date_time(row[0])
                launch_dict['Date'].append(datetime_list[0])
                launch_dict['Time'].append(datetime_list[1])

                # Booster Version
                bv = booster_version(row[1])
                launch_dict['Version Booster'].append(bv)

                # Launch Site
                launch_site = row[2]

Key 'Date and time ( )' not found in column names.


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def date_time(element):
    return [el.strip(',') for el in element.text.split(' ')]

def booster_version(element):
    if element.a:
        return element.a.string
    else:
        return None

def get_mass(element):
    mass_text = element.text.strip()
    if mass_text:
        return mass_text.replace('kg', '').replace(',', '')  # Remove commas as well
    else:
        return None

def landing_status(element):
    try:
        landing_text = element.find('div', class_='mw-collapsible-text').text.strip()
        return landing_text
    except AttributeError:
        return None

static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    html_tables = soup.find_all('table', "wikitable plainrowheaders collapsible")

    column_names = []
    for th in html_tables[2].find_all('th'): #Extract from the third table only
        name = th.text.strip()
        if name:
            column_names.append(name)
    launch_dict = dict.fromkeys(column_names)
    try:
        del launch_dict['Date and time ( )']
    except KeyError:
        pass

    launch_dict['Flight No.'] = []
    launch_dict['Launch site'] = []
    launch_dict['Payload'] = []
    launch_dict['Payload mass'] = []
    launch_dict['Orbit'] = []
    launch_dict['Customer'] = []
    launch_dict['Launch outcome'] = []
    launch_dict['Version Booster'] = []
    launch_dict['Booster landing'] = []
    launch_dict['Date'] = []
    launch_dict['Time'] = []

    extracted_row = 0
    for table_number, table in enumerate(html_tables):
        if table_number == 2: #Extract from the third table only
            for rows in table.find_all("tr"):
                if rows.th:
                    if rows.th.string:
                        flight_number = rows.th.string.strip()
                        if flight_number.isdigit():
                            flag = True
                        else:
                            flag = False
                    else:
                        flag = False
                else:
                    flag = False

                row = rows.find_all('td')
                if flag:
                    extracted_row += 1
                    try: #Added try except blocks to handle potential index errors
                        launch_dict['Flight No.'].append(flight_number)
                        datetime_list = date_time(row[0])
                        launch_dict['Date'].append(datetime_list[0])
                        launch_dict['Time'].append(datetime_list[1])
                        bv = booster_version(row[1])
                        launch_dict['Version Booster'].append(bv)
                        launch_dict['Launch site'].append(row[2].a.string)
                        launch_dict['Payload'].append(row[3].a.string)
                        launch_dict['Payload mass'].append(get_mass(row[4]))
                        launch_dict['Orbit'].append(row[5].a.string)
                        launch_dict['Customer'].append(row[6].a.string)
                        launch_dict['Launch outcome'].append(list(row[7].strings)[0])
                        launch_dict['Booster landing'].append(landing_status(row[8]))
                    except IndexError:
                        print(f"IndexError at row {extracted_row}. Skipping row.")

    df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})
    print(df.head())
    print(df.info())
else:
    print(f"Error: Failed to retrieve page. Status code: {response.status_code}")

In [19]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
